## INITIALIZE LOGGER

In [ ]:
import wandb
wandb.login(key="8dd8cac018b63a0686dd945ef666c79145af226b")

## IMPORTS

In [ ]:
import json
import math
from collections import OrderedDict
import torch
from torch import nn, Tensor
from typing import Union, Tuple, List, Iterable, Dict
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.optim import AdamW
from torch.utils.data import DataLoader
import numpy as np
import gzip, csv
import pandas as pd
from tqdm.auto import tqdm
import torch.nn.init as init
torch.manual_seed(0)
np.random.seed(0)
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import argparse
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from torch import nn, Tensor
import torch.nn.functional as F
from torch.utils.data import Dataset
from avalanche.benchmarks.utils import AvalancheDataset
from avalanche.benchmarks.generators import nc_benchmark
from transformers import BertTokenizer
from datasets import load_dataset
import re
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.models import MTSimpleMLP
from avalanche.training.supervised import EWC
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics
from avalanche.logging import InteractiveLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics
from avalanche.training.templates import SupervisedTemplate
from avalanche.training.plugins import ReplayPlugin, EWCPlugin
from avalanche.logging import WandBLogger
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# MODEL

## ACTIVATION FUNCTION

In [ ]:
def gelu(x):
    """Implementation of the gelu activation function."""
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))


## POSTIONAL ENCODING LAYER

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, embed_dim: int, drop_rate=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=drop_rate)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2) * (-math.log(10000.0) / embed_dim))
        pe = torch.zeros(1, max_len, embed_dim)
        pe[0, :, 0::2] = torch.sin(position * div_term)
        pe[0, :, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Args:
            x: Tensor, shape [batch_size, seq_len, embedding_dim]
        
        Returns:
            torch.Tensor: Output tensor after adding positional encodings and applying dropout.
                 It has the same shape as the input tensor [batch_size, seq_len, embedding_dim].
                 The positional encodings are added to the input tensor along the sequence length dimension,
                 and dropout is applied to the combined tensor.
        
        """
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

## ATTENTION MECHANISM

In [ ]:
def scaled_dot_product(q, k, v, attn_drop_rate=0.1):
    """
    Args:
      q: query, shape: (batch, # heads, seq len, head dimension)
      k: keys, shape: (batch, # heads, seq len, head dimension)
      v: value, shape: (batch, # heads, seq len, head dimension)
      attn_drop_rate: probability of an element to be zeroed,
      mask: the optional masking of specific entries in the attention matrix.
              shape: (batch, seq len)
    
     Returns:
        torch.Tensor: Output tensor after scaled dot product attention computation.
           Shape: (batch, # heads, seq len, head dimension).
    
    """

    d_k = q.shape[-1]
    attn_logits = torch.matmul(q, k.transpose(-1, -2))
    attn_logits = attn_logits/math.sqrt(d_k)
    attention = F.softmax(attn_logits, dim=-1)
    attention = F.dropout(attention, p=attn_drop_rate)
    values = torch.matmul(attention,v)
    return values

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, n_heads, attn_drop_rate):
        super().__init__()
        self.embed_dim = embed_dim
        self.n_heads = n_heads
        self.head_dim = embed_dim // n_heads
        self.attn_drop_rate = attn_drop_rate
        self.query = nn.Linear(self.embed_dim, self.n_heads*self.head_dim)
        self.key = nn.Linear(self.embed_dim, self.n_heads*self.head_dim)
        self.value = nn.Linear(self.embed_dim, self.n_heads*self.head_dim)
        self.o_proj = nn.Linear(self.embed_dim, self.n_heads*self.head_dim)
        self._reset_parameters()

    def _reset_parameters(self):
      nn.init.xavier_uniform_(self.query.weight)
      self.query.bias.data.fill_(0)
      nn.init.xavier_uniform_(self.key.weight)
      self.key.bias.data.fill_(0)
      nn.init.xavier_uniform_(self.value.weight)
      self.value.bias.data.fill_(0)
      nn.init.xavier_uniform_(self.o_proj.weight)
      self.o_proj.bias.data.fill_(0)

    def split_heads(self, tensor):
       new_shape = tensor.size()[:-1] + (self.n_heads, self.head_dim)
       tensor = tensor.view(*new_shape)
       tensor = tensor.permute(0, 2, 1, 3).contiguous()
       return tensor

    def merge_heads(self, tensor, batch_size, seq_length):
       tensor = tensor.transpose(1, 2).contiguous().view(batch_size, seq_length, self.embed_dim)
       return tensor

    def forward(self, embedding):
      """
       Args:
        embedding (torch.Tensor): 
            A tensor of shape (batch_size, seq_length, embed_dim) representing the input embeddings.
            - `batch_size`: The number of samples in the batch.
            - `seq_length`: The number of tokens (or time steps) in each sequence.
            - `embed_dim`: The dimension of the embedding for each token.
       
       Returns:
        torch.Tensor: 
            A tensor of shape (batch_size, seq_length, embed_dim) representing the attended embeddings.
            - `batch_size`: The number of samples in the batch.
            - `seq_length`: The number of tokens (or time steps) in each sequence.
            - `embed_dim`: The dimension of the embedding for each token.
      """
      batch_size, seq_length, embed_dim = embedding.size()
      q, k, v = self.query(embedding), self.key(embedding), self.value(embedding)
      q = self.split_heads(q)
      k = self.split_heads(k)
      v = self.split_heads(v)
      values = scaled_dot_product(q, k, v, self.attn_drop_rate)
      values = self.merge_heads(values, batch_size, seq_length)
      attended_embeds = self.o_proj(values)
      return attended_embeds

## LAYER NORMALIZATION LAYER

In [ ]:
class LayerNormalization(nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape=parameters_shape
        self.eps=eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape))
        self.beta =  nn.Parameter(torch.zeros(parameters_shape))

    def forward(self, inputs):
        """
         Args:
         inputs (Tensor): Input tensor to normalize.
         
         Returns:
                torch.Tensor: Normalized tensor after applying layer normalization.
                 It has the same shape as the input tensor `(batch_size, *parameters_shape)`.

        """
        dims = [-(i + 1) for i in range(len(self.parameters_shape))]
        mean = inputs.mean(dim=dims, keepdim=True)
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True)
        std = (var + self.eps).sqrt()
        y = (inputs - mean) / std
        out = self.gamma * y  + self.beta
        return out

## FEEDFORWARD LAYER

In [ ]:
class PositionwiseFeedForward(nn.Module): 

    def __init__(self, embed_dim, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(embed_dim, 4*embed_dim)
        self.linear2 = nn.Linear(4*embed_dim, embed_dim)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        """
         Args:
             x (torch.Tensor): Input tensor to the feedforward network.
                 Its shape should be `(batch_size, sequence_length, embed_dim)`.
                 `batch_size` is the number of sequences in a batch,
                 `sequence_length` is the length of each sequence,
                 and `embed_dim` is the dimensionality of the input and output embeddings.
     
         Returns:
             torch.Tensor: Output tensor of the feedforward network.
                 It has the same shape as the input tensor `(batch_size, sequence_length, embed_dim)`.
        
        """
        x = self.linear1(x)
        x = self.gelu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

## CLASSIFIER LAYER

In [ ]:
class Classifier(nn.Module):
    def __init__(self, input_dim, numclasses, dropout_rate=0.1):
        super(Classifier, self).__init__()
        self.linear = nn.Linear(input_dim, numclasses) 

    def forward(self, x):
     """
        Args:
            x (torch.Tensor): Input tensor to the classifier.
                Its shape should be `()`.
                `batch_size` is the number of samples in the batch,
                and `input_dim` is the dimensionality of the input features.

        Returns:
            torch.Tensor: Output tensor representing the logits for each class.
                It has the shape `()`.
                `batch_size` is the number of samples in the batch,
                and `num_classes` is the number of classes in the classification task.
     """
     x = self.linear(x)
     return x

## ENCODER LAYER

In [ ]:
class EncoderLayer(nn.Module):

    def __init__(self, embed_dim, n_heads, attn_drop_rate, layer_drop_rate):
        super(EncoderLayer, self).__init__()
        self.embed_dim = embed_dim
        self.n_heads = n_heads
        self.attention = MultiHeadAttention(self.embed_dim, self.n_heads, attn_drop_rate)
        self.norm1 = LayerNormalization(parameters_shape=[self.embed_dim])
        self.dropout1 = nn.Dropout(p=layer_drop_rate)
        self.ffn = PositionwiseFeedForward(self.embed_dim,layer_drop_rate)
        self.norm2 = LayerNormalization(parameters_shape=[self.embed_dim])
        self.dropout2 = nn.Dropout(p=layer_drop_rate)

    def forward(self, x):
     """
        Args:
        x (torch.Tensor): Input tensor to the encoder layer.
            Its shape should be `(batch_size, seq_length, embed_dim)`.
            - `batch_size`: The number of samples in the batch.
            - `seq_length`: The number of tokens (or time steps) in each sequence.
            - `embed_dim`: The dimension of the embedding for each token.
            
        Returns:
            torch.Tensor: Output tensor representing the encoded representations.
                It has the same shape as the input tensor `(batch_size, seq_length, embed_dim)`.
                - `batch_size`: The number of samples in the batch.
                - `seq_length`: The number of tokens (or time steps) in each sequence.
                - `embed_dim`: The dimension of the embedding for each token.
            
     """
     residual_x = x
     x = self.attention(x)
     x = self.dropout1(x)
     x = x + residual_x
     x = self.norm1(x)
     residual_x = x
     x = self.ffn(x)
     x = self.dropout2(x)
     x = x + residual_x
     x = self.norm2(x)
     return x

## TRANSFORMER LAYER

In [ ]:
class ENCTransformer(nn.Module): 
    def __init__(self, n_layers, vocab_size, embed_dim, n_heads, num_classes, attn_drop_rate, layer_drop_rate):
        super().__init__()
        self.embed = nn.Embedding(vocab_size+1, embed_dim)
        self.position = PositionalEncoding(embed_dim, layer_drop_rate)
        self.net = nn.Sequential(*[
        EncoderLayer(embed_dim, n_heads, attn_drop_rate, layer_drop_rate) for _ in range(n_layers)
        ])
        self.pooler = nn.Sequential(OrderedDict([
            ('dense', nn.Linear(embed_dim, embed_dim)),
            ('activation', nn.Tanh()),
        ]))
        self.classifier = Classifier(embed_dim, num_classes)
        self.saved_sample = None

    def forward(self, batch_text):
     """
        Args:
            batch_text (torch.Tensor): Batch of input texts represented as token indices.
                Its shape should be `(batch_size, seq_length)`.
        Returns:
            torch.Tensor: Predicted logits for each class.
                It has the shape `(batch_size, num_classes)`.
                - `batch_size`: The number of samples in the batch.
                - `num_classes`: The number of classes in the classification task.
     """
     batch_text = batch_text.squeeze(1)
     embedding = self.position(self.embed(batch_text)) 
     new_embedding = self.net((embedding))
     o = self.pooler(new_embedding[:, 0])
     preds = self.classifier(o)
     return preds



## DATALOADER & BENCHMARK

In [ ]:
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        """
        Args:
            texts (list of str): List of text samples.
            labels (list of int): List of labels corresponding to the text samples.
        """
        self.texts = texts
        self.labels = labels
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        #self.tokenizer = BertTokenizer(vocab_file='/kaggle/input/custom-dataset-for-ncoder/custom_vocab.txt')
    def __len__(self):
        return len(self.texts)

    @staticmethod
    def preprocess_text(text):
        text = text.lower()
        text = re.sub(r'http\S+|www.\S+', ' ', text)
        text = re.sub(r'\S*@\S*\s?', ' ', text)
        text = re.sub(r'[^a-z0-9,.!? ]', ' ', text)
        return text

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        text = TextDataset.preprocess_text(text)
        encoded_text = self.tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=512)
        encoded_text = encoded_text["input_ids"]

        return encoded_text, label

dataset = load_dataset("setfit/20_newsgroups")

train_text = dataset['train']['text']
train_labels = dataset['train']['label']

test_text = dataset['test']['text']
test_labels = dataset['test']['label']

train_data = TextDataset(train_text, train_labels)
test_data = TextDataset(test_text, test_labels)

avl_train_data = AvalancheDataset(train_data)
avl_test_data = AvalancheDataset(test_data)


avl_train_data.targets = train_labels
avl_test_data.targets = test_labels

benchmark = nc_benchmark(
    test_dataset=avl_test_data,  
    train_dataset=avl_train_data,
    n_experiences=5,  
    task_labels=False  
)

train_stream = benchmark.train_stream
test_stream = benchmark.test_stream
experience = train_stream[0]

t_label = experience.task_label
dataset = experience.dataset

## CONFIG

In [ ]:
embed_dim = 512
n_heads = 8
n_layers = 4
vocab_size = 30522
#vocab_size = 91015
attn_drop_rate = 0.5
layer_drop_rate = 0.2
num_classes=20
num_epochs = 40
batch_size = 32
model = ENCTransformer(n_layers, vocab_size, embed_dim, n_heads, num_classes, attn_drop_rate, layer_drop_rate)
model = model.to(device)

def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, dim=1)
    correct = (predicted == labels).sum().item()  
    total = labels.size(0) 
    accuracy = correct / total
    return accuracy


optimizer = AdamW(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5)

## TRAINING TESTING LOOP

In [ ]:
interactive_logger = InteractiveLogger()
loggers=[]
loggers.append(InteractiveLogger())
loggers.append(WandBLogger(project_name="sequential_meta_classifier", run_name="40e_0.0001lr_512dim_4h_3l_0.2adr_0.2ldr_AVL_COS_NOMETA"))
eval_plugin = EvaluationPlugin(
    accuracy_metrics(
        minibatch=False, epoch=True, experience=True, stream=True
    ),
    confusion_matrix_metrics(num_classes=benchmark.n_classes, save_image=False,
                             stream=True),
    forgetting_metrics(experience=True, stream=True),
    loggers=loggers,
)

replay = ReplayPlugin(mem_size=1000)
ewc = EWCPlugin(ewc_lambda=1)


cl_strategy = SupervisedTemplate(
    model, optimizer,
    criterion, train_mb_size=batch_size, train_epochs=num_epochs, eval_mb_size=batch_size,
    evaluator=eval_plugin,device=device,plugins=[replay] )


# strategy = EWC(
#     model=model,
#     optimizer=optimizer,
#     criterion=criterion,
#     train_mb_size=batch_size,
#     train_epochs=num_epochs,
#     eval_mb_size=batch_size,
#     device=device,
#     evaluator=eval_plugin,
#     ewc_lambda=0.4,
# )

In [ ]:
for train_task in train_stream:
    cl_strategy.train(train_task)
    cl_strategy.eval(test_stream)